## E-Commerce Customer Segmentation

### Project Overview
* Customer segmentation using different clustering algorithms


### Code and Resources Used 
**Python Version:** 3.7  
**Packages:** pandas, numpy, matplotlib, seaborn, sklearn.

### Dataset
The dataset is taken from The UCI Machine Learning Repository (https://archive.ics.uci.edu/)  
Dataset: https://archive.ics.uci.edu/dataset/352/online+retail 

### EDA
* Features grouping and distribution analysis.
* Continuous variables analysis against different groups.
* Correlation analysis

Below are notebook captures as examples of our analysis:  

### Data preparation


### Notebook content:

1. Exploratory Data Analysis
2. Data Preparation
3. Models Training, Evaluation and Error Analysis
4. Conclusion

### 1. Exploratory Data Analysis
---

In [1]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

import seaborn as sns

In [2]:
data = pd.read_csv('dataset/data.csv', encoding="ISO-8859-1")
# data_test = pd.read_csv('dataset/data.csv', encoding="ISO-8859-1", dtype={'CustomerID': str,'InvoiceID': str})

This is the data information we have:

    InvoiceNo    ID       Categorical   6-digit integral number uniquely assigned to each transaction. Start with 'c', indicates a cancellation  
    StockCode    ID       Categorical   5-digit integral number uniquely assigned to each distinct product  
    Description  Feature  Categorical   product name  
    Quantity     Feature  Integer       the quantities of each product (item) per transaction  
    InvoiceDate  Feature  Date          the day and time when each transaction was generated  
    UnitPrice    Feature  Continuous    product price (sterlin) per unit	  
    CustomerID   Feature  Categorical   5-digit integral number uniquely assigned to each customer  
    Country      Feature  Categorical   the name of the country where each customer resides  

In [3]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


Before starting filtering the dataset we will make a security copy:

In [4]:
df = data.copy()

Duplicates inspection:

In [58]:
df.duplicated().sum()

5268

There are 5269 duplicated rows. Let's visualize them:

In [61]:
df_dups = df[df.duplicated(keep=False)]
df_dups = df_dups.sort_values(by=['InvoiceNo', 'StockCode', 'Description', 'CustomerID', 'Quantity'])
df_dups.head(6)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
494,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,12/1/2010 11:45,1.25,17908.0,United Kingdom
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,12/1/2010 11:45,1.25,17908.0,United Kingdom
485,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,12/1/2010 11:45,4.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,12/1/2010 11:45,4.95,17908.0,United Kingdom
489,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,12/1/2010 11:45,2.10,17908.0,United Kingdom
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,12/1/2010 11:45,2.10,17908.0,United Kingdom


In [5]:
df = df.drop_duplicates()

Null values inspection:

In [63]:
df.isnull().sum()

InvoiceNo      0     
StockCode      0     
Description    1454  
Quantity       0     
InvoiceDate    0     
UnitPrice      0     
CustomerID     135037
Country        0     
dtype: int64

There are null values in 'Description' and in 'CustomerID'. 

In [6]:
df[df['Description'].isnull() & df['CustomerID'].isnull()].shape[0]

1454

The rows with null values in 'Description' have also null values in 'Customer ID' feature. Let's confirm this with some examples:

In [65]:
df[df['Description'].isnull() & df['CustomerID'].isnull()].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,12/1/2010 14:32,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,12/1/2010 14:34,0.0,NaN,United Kingdom


It's not a good choice to impute 'Customer ID' null values so we'll delete them:

In [7]:
df = df.dropna()

In [8]:
df.shape

(401604, 8)

### Find numerical variables (continuous, discrete, target)

In [4]:
numerical = [var for var in data.columns if data[var].dtype!='O']
print('There are {} numerical variables:'.format(len(numerical)))
numerical

There are 3 numerical variables:


['Quantity', 'UnitPrice', 'CustomerID']

### Find discrete variables

We will select the ones that contain a finite and small number of distinct values from all the numerical ones:

In [5]:
discrete = []

for var in numerical:
    if len(data[var].unique()) < 20:
        print(var, 'values: ', data[var].unique())
        discrete.append(var)
print()
print('There are {} discrete variables'.format(len(discrete)))


There are 0 discrete variables


### Continuous variables

In [7]:
continuous = [var for var in numerical if var not in discrete and var not in ['price']]

print('There are {} numerical and continuous variables'.format(len(continuous)))
continuous

There are 3 numerical and continuous variables


['Quantity', 'UnitPrice', 'CustomerID']

### Find categorical variables 

In [8]:
categorical = [var for var in data.columns if data[var].dtype=='O']  

print('There are {} categorical variables:'.format(len(categorical)))
categorical

There are 5 categorical variables:


['InvoiceNo', 'StockCode', 'Description', 'InvoiceDate', 'Country']

Categorical variables inspection:

In [9]:
pd.set_option('display.max_colwidth', 0)
data[categorical].agg(['unique']).transpose()

,unique
InvoiceNo,"[536365, 536366, 536367, 536368, 536369, 536370, 536371, 536372, 536373, 536374, 536375, 536376, 536377, 536378, 536380, 536381, C536379, 536382, C536383, 536384, 536385, 536386, 536387, 536388, 536389, 536390, C536391, 536392, 536393, 536394, 536395, 536396, 536397, 536398, 536399, 536400, 536401, 536402, 536403, 536404, 536405, 536406, 536407, 536408, 536409, 536412, 536414, 536415, 536416, 536420, 536423, 536425, 536437, 536446, 536460, 536463, 536464, 536466, 536477, 536488, 536500, 536502, 536508, C536506, 536514, 536520, 536521, 536522, 536523, 536524, 536525, 536526, 536527, 536528, 536529, 536530, 536531, 536532, 536533, 536534, 536535, 536536, 536537, 536538, 536539, 536540, 536541, 536542, C536543, 536544, 536545, 536546, 536547, C536548, 536549, 536550, 536551, 536552, 536553, 536554, ...]"
StockCode,"[85123A, 71053, 84406B, 84029G, 84029E, 22752, 21730, 22633, 22632, 84879, 22745, 22748, 22749, 22310, 84969, 22623, 22622, 21754, 21755, 21777, 48187, 22960, 22913, 22912, 22914, 21756, 22728, 22727, 22726, 21724, 21883, 10002, 21791, 21035, 22326, 22629, 22659, 22631, 22661, 21731, 22900, 21913, 22540, 22544, 22492, POST, 22086, 20679, 37370, 21871, 21071, 21068, 82483, 82486, 82482, 82494L, 21258, 22114, 21733, 22386, 85099C, 21033, 20723, 84997B, 84997C, 21094, 20725, 21559, 22352, 21212, 21975, 21977, 84991, 84519A, 85183B, 85071B, 21931, 21929, 22961, 22139, 84854, 22411, 82567, 21672, 22774, 22771, 71270, 22262, 22637, 21934, 21169, 21166, 21175, 37444A, 37444C, 22083, 84971S, 47580, 22261, 84832, ...]"
Description,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE METAL LANTERN, CREAM CUPID HEARTS COAT HANGER, KNITTED UNION FLAG HOT WATER BOTTLE, RED WOOLLY HOTTIE WHITE HEART., SET 7 BABUSHKA NESTING BOXES, GLASS STAR FROSTED T-LIGHT HOLDER, HAND WARMER UNION JACK, HAND WARMER RED POLKA DOT, ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHOUSE BEDROOM , POPPY'S PLAYHOUSE KITCHEN, FELTCRAFT PRINCESS CHARLOTTE DOLL, IVORY KNITTED MUG COSY , BOX OF 6 ASSORTED COLOUR TEASPOONS, BOX OF VINTAGE JIGSAW BLOCKS , BOX OF VINTAGE ALPHABET BLOCKS, HOME BUILDING BLOCK WORD, LOVE BUILDING BLOCK WORD, RECIPE BOX WITH METAL HEART, DOORMAT NEW ENGLAND, JAM MAKING SET WITH JARS, RED COAT RACK PARIS FASHION, YELLOW COAT RACK PARIS FASHION, BLUE COAT RACK PARIS FASHION, BATH BUILDING BLOCK WORD, ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELIKE RED , ALARM CLOCK BAKELIKE GREEN, PANDA AND BUNNIES STICKER SHEET, STARS GIFT TAPE , INFLATABLE POLITICAL GLOBE , VINTAGE HEADS AND TAILS CARD GAME , SET/2 RED RETROSPOT TEA TOWELS , ROUND SNACK BOXES SET OF4 WOODLAND , SPACEBOY LUNCH BOX , LUNCH BOX I LOVE LONDON, CIRCUS PARADE LUNCH BOX , CHARLOTTE BAG DOLLY GIRL DESIGN, RED TOADSTOOL LED NIGHT LIGHT, SET 2 TEA TOWELS I LOVE LONDON , VINTAGE SEASIDE JIGSAW PUZZLES, MINI JIGSAW CIRCUS PARADE , MINI JIGSAW SPACEBOY, MINI PAINT SET VINTAGE , POSTAGE, PAPER CHAIN KIT 50'S CHRISTMAS , EDWARDIAN PARASOL RED, RETRO COFFEE MUGS ASSORTED, SAVE THE PLANET MUG, VINTAGE BILLBOARD DRINK ME MUG, VINTAGE BILLBOARD LOVE/HATE MUG, WOOD 2 DRAWER CABINET WHITE FINISH, WOOD S/3 CABINET ANT WHITE FINISH, WOODEN PICTURE FRAME WHITE FINISH, WOODEN FRAME ANTIQUE WHITE , VICTORIAN SEWING BOX LARGE, HOT WATER BOTTLE TEA AND SYMPATHY, RED HANGING HEART T-LIGHT HOLDER, JUMBO BAG PINK POLKADOT, JUMBO BAG BAROQUE BLACK WHITE, JUMBO BAG CHARLIE AND LOLA TOYS, STRAWBERRY CHARLOTTE BAG, RED 3 PIECE RETROSPOT CUTLERY SET, BLUE 3 PIECE POLKADOT CUTLERY SET, SET/6 RED SPOTTY PAPER PLATES, LUNCH BAG RED RETROSPOT, STRAWBERRY LUNCH BOX WITH CUTLERY, LUNCH BOX WITH CUTLERY RETROSPOT , PACK OF 72 RETROSPOT CAKE CASES, PACK OF 60 DINOSAUR CAKE CASES, PACK OF 60 PINK PAISLEY CAKE CASES, 60 TEATIME FAIRY CAKE CASES, TOMATO CHARLIE+LOLA COASTER SET, CHARLIE & LOLA WASTEPAPER BIN FLORA, RED CHARLIE+LOLA PERSONAL DOORSIGN, JUMBO STORAGE BAG SUKI, JUMBO BAG PINK VINTAGE PAISLEY, JAM MAKING SET PRINTED, RETROSPOT TEA SET CERAMIC 11 PC , GIRLY PINK TOOL SET, JUMBO SHOPPER VINTAGE 

Let's inspect numerical variables distributions:

In [9]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,401604.000000,401604.000000,401604.000000
mean,12.183273,3.474064,15281.160818
std,250.283037,69.764035,1714.006089
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13939.000000
50%,5.000000,1.950000,15145.000000
75%,12.000000,3.750000,16784.000000
max,80995.000000,38970.000000,18287.000000


Cancelled orders have a 'InvoiceNo' code that starts with a 'C' and negative 'Quantity' values:

In [77]:
df[df['Quantity']<0].shape[0]

8872

In [78]:
cancelations = 0;

for i in df['InvoiceNo']:
    if i.startswith("C"):
        cancelations+=1
        
print(f'Total cancelations: {cancelations}')        

Total cancelations: 8872


We delete those choosing only postitive values for 'Quantity':

In [79]:
df = df[df['Quantity']>0]

In [82]:
df.shape

(392732, 8)

In [83]:
data.shape

(541909, 8)

In [84]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [90]:
df[df['Description']=='']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
